# 词向量详解

## 📋 文档说明

本文档是词向量的详细理论讲解，比父目录的《文本特征提取详解》更加深入和详细。本文档将深入讲解词向量的原理、数学推导和实现细节。通过本文档，你将能够：

1. **深入理解词向量的原理**：从one-hot编码到分布式表示的完整流程
2. **掌握Word2Vec的数学原理**：理解Skip-gram和CBOW的数学定义、为什么有效、如何优化
3. **理解词向量的训练方法**：理解负采样、层次softmax等优化技术
4. **掌握词向量在VLA中的应用**：理解词向量在VLA模型中的具体应用和优势

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过可视化图表和数学推导更好地理解词向量的原理和过程。

**文档结构**：
- 父目录：文本特征提取详解（见../文本特征提取详解.ipynb）
- 本文档：词向量详解（本文档）

---

## 📚 术语表（按出现顺序）

### 1. 词向量 (Word Embedding)
- **中文名称**：词向量
- **英文全称**：Word Embedding
- **定义**：词向量是指将词语映射到固定维度的数值向量的方法，是自然语言处理的基础技术。词向量能够捕获词语的语义信息，语义相似的词语在向量空间中距离较近。词向量的训练方法包括Word2Vec、GloVe、FastText等。Word2Vec通过预测上下文词语或中心词语来训练词向量，GloVe通过统计词语共现矩阵来训练词向量，FastText通过子词（subword）来训练词向量。词向量的优势在于能够将离散的词语转换为连续的数值向量，便于神经网络处理。词向量在VLA中的应用包括将语言指令中的词语转换为向量表示，这些向量将被用于后续的语言编码和理解。虽然现代VLA模型通常使用预训练的语言模型（如BERT、GPT），但理解词向量的原理有助于理解语言编码器的工作原理。词向量是语言理解的基础，它提供了词语的数值表示，使得计算机能够处理和理解自然语言。
- **核心组成**：词向量的核心组成包括：1）词汇表：建立词语到索引的映射，将离散的词语转换为索引；2）词向量矩阵：存储每个词语的向量表示，是一个$V \times d$的矩阵，其中$V$是词汇表大小，$d$是向量维度；3）训练方法：使用不同的训练方法学习词向量，如Word2Vec的Skip-gram和CBOW、GloVe的全局统计、FastText的子词模型；4）向量维度：确定词向量的维度，通常为100-300维，维度越高，表达能力越强，但计算成本也越高；5）预训练词向量：使用大规模文本数据预训练词向量，如维基百科、新闻语料等；6）微调：在特定任务上微调词向量，使其更适合特定任务。词向量的训练通常使用大规模文本数据，通过无监督学习的方式学习词语的语义表示。
- **在VLA中的应用**：在VLA中，词向量用于将语言指令中的词语转换为向量表示。虽然现代VLA模型通常使用预训练的语言模型（如BERT、GPT），这些模型内部也使用词向量，但词向量通常是可学习的，在训练过程中不断更新。理解词向量的原理有助于理解语言编码器如何将文本转换为数值表示，如何捕获词语的语义信息。在某些VLA应用中，词向量仍然有用，例如用于简单的文本分类、文本相似度计算等任务。词向量的语义信息能够帮助VLA模型理解语言指令中词语的含义，从而更好地理解指令的意图。
- **相关概念**：Word2Vec、GloVe、FastText、语言编码器、文本特征提取、分布式表示
- **首次出现位置**：本文档标题
- **深入学习**：参考父目录的[文本特征提取详解](../文本特征提取详解.ipynb)
- **直观理解**：想象词向量就像给每个词语制作一张"身份证"，这张身份证用数字编码了这个词语的含义。语义相似的词语，它们的"身份证"数字也相似。例如，"猫"和"狗"都是动物，它们的词向量在向量空间中距离较近；"猫"和"汽车"不是同类，它们的词向量在向量空间中距离较远。在VLA中，词向量帮助模型理解语言指令中词语的含义，从而生成相应的动作。词向量就像词语的"数字指纹"，能够唯一标识词语的语义特征。

### 2. Word2Vec
- **中文名称**：Word2Vec
- **英文全称**：Word2Vec
- **定义**：Word2Vec是一种用于训练词向量的方法，由Google在2013年提出。Word2Vec通过预测上下文词语或中心词语来训练词向量，基于"相似上下文的词语具有相似语义"的假设。Word2Vec有两种训练方法：Skip-gram和CBOW。Skip-gram使用中心词语预测上下文词语，适合处理罕见词语；CBOW使用上下文词语预测中心词语，训练速度更快。Word2Vec的优势在于：1）简单高效：训练方法简单，计算效率高；2）语义捕获：能够捕获词语的语义信息，语义相似的词语在向量空间中距离较近；3）可扩展性：可以处理大规模文本数据；4）预训练模型：可以使用预训练的Word2Vec模型。Word2Vec是词向量训练的基础方法，虽然现代VLA模型通常使用更先进的语言模型，但理解Word2Vec的原理有助于理解语言编码器的工作原理。
- **核心组成**：Word2Vec的核心组成包括：1）Skip-gram模型：使用中心词语预测上下文词语，最大化上下文词语的条件概率；2）CBOW模型：使用上下文词语预测中心词语，最大化中心词语的条件概率；3）负采样：使用负采样技术加速训练，避免计算所有词语的softmax；4）层次softmax：使用层次softmax技术加速训练，将softmax计算复杂度从$O(V)$降低到$O(\log V)$；5）训练目标：最大化词语共现的概率，使语义相似的词语在向量空间中距离较近；6）优化方法：使用随机梯度下降（SGD）或Adam优化器训练模型。Word2Vec的训练通常使用大规模文本数据，通过无监督学习的方式学习词语的语义表示。
- **在VLA中的应用**：在VLA中，Word2Vec用于训练词向量，将语言指令中的词语转换为向量表示。虽然现代VLA模型通常使用预训练的语言模型（如BERT、GPT），这些模型内部也使用类似Word2Vec的方法，但词向量通常是可学习的，在训练过程中不断更新。理解Word2Vec的原理有助于理解语言编码器如何将文本转换为数值表示，如何捕获词语的语义信息。Word2Vec的语义信息能够帮助VLA模型理解语言指令中词语的含义，从而更好地理解指令的意图。在某些VLA应用中，Word2Vec仍然有用，例如用于简单的文本分类、文本相似度计算等任务。
- **相关概念**：Skip-gram、CBOW、负采样、层次softmax、词向量、分布式表示
- **首次出现位置**：本文档第1.1节
- **深入学习**：参考本文档的Word2Vec详细讲解部分
- **直观理解**：想象Word2Vec就像一位"语言学习者"，他通过观察词语在文本中的出现模式来学习词语的含义。例如，看到"猫"和"狗"经常出现在相似的上下文中（如"宠物"、"动物"），Word2Vec就会认为它们语义相似，将它们放在向量空间中相近的位置。在VLA中，Word2Vec帮助模型理解语言指令中词语的含义，从而生成相应的动作。Word2Vec就像词语的"语义地图"，能够将语义相似的词语映射到相近的位置。

### 3. Skip-gram
- **中文名称**：Skip-gram
- **英文全称**：Skip-gram
- **定义**：Skip-gram是Word2Vec的一种训练方法，使用中心词语预测上下文词语。Skip-gram的目标是最大化上下文词语的条件概率，即给定中心词语，预测其上下文词语的概率。Skip-gram的数学表示为：$P(w_{t-k}, \ldots, w_{t+k} | w_t)$，其中$w_t$是中心词语，$w_{t-k}$到$w_{t+k}$是上下文词语。Skip-gram的优势在于：1）适合罕见词语：能够更好地处理罕见词语，因为每个中心词语都有多个上下文词语；2）训练效率：训练过程相对简单，计算效率较高；3）语义捕获：能够捕获词语的语义信息，语义相似的词语在向量空间中距离较近。Skip-gram是Word2Vec的核心方法之一，在词向量训练中取得了很好的效果。
- **核心组成**：Skip-gram的核心组成包括：1）输入层：将中心词语转换为one-hot编码；2）词嵌入层：将one-hot编码映射到词向量；3）输出层：使用词向量预测上下文词语的概率；4）损失函数：使用交叉熵损失函数，最大化上下文词语的条件概率；5）负采样：使用负采样技术加速训练，避免计算所有词语的softmax；6）优化方法：使用随机梯度下降（SGD）或Adam优化器训练模型。Skip-gram的训练通常使用大规模文本数据，通过无监督学习的方式学习词语的语义表示。
- **在VLA中的应用**：在VLA中，Skip-gram用于训练词向量，将语言指令中的词语转换为向量表示。虽然现代VLA模型通常使用预训练的语言模型（如BERT、GPT），这些模型内部也使用类似Skip-gram的方法，但词向量通常是可学习的，在训练过程中不断更新。理解Skip-gram的原理有助于理解语言编码器如何将文本转换为数值表示，如何捕获词语的语义信息。Skip-gram的语义信息能够帮助VLA模型理解语言指令中词语的含义，从而更好地理解指令的意图。
- **相关概念**：Word2Vec、CBOW、负采样、词向量、上下文预测
- **首次出现位置**：本文档第2.1节
- **深入学习**：参考本文档的Skip-gram详细讲解部分
- **直观理解**：想象Skip-gram就像一位"猜谜游戏"的玩家，他看到一个词语（中心词语），然后猜测这个词语周围可能出现的其他词语（上下文词语）。例如，看到"猫"，Skip-gram会猜测周围可能出现"宠物"、"动物"、"毛茸茸"等词语。通过这种"猜谜"的方式，Skip-gram学会了词语的语义关系。在VLA中，Skip-gram帮助模型理解语言指令中词语的含义，从而生成相应的动作。

---

## 📋 概述

### 什么是词向量

词向量是指将词语映射到固定维度的数值向量的方法，是自然语言处理的基础技术。词向量能够捕获词语的语义信息，语义相似的词语在向量空间中距离较近。

### 为什么重要

词向量对于VLA学习非常重要，原因包括：

1. **语言理解的基础**：词向量是语言理解的基础，提供了词语的数值表示
2. **语义捕获**：能够捕获词语的语义信息，语义相似的词语在向量空间中距离较近
3. **计算效率**：将离散的词语转换为连续的数值向量，便于神经网络处理
4. **预训练方法**：可以使用预训练的词向量，提高特征提取质量

### 学习目标

通过本文档的学习，你将能够：

1. **深入理解词向量**：理解词向量的原理和方法
2. **掌握Word2Vec**：理解Skip-gram和CBOW的数学定义和计算方法
3. **理解训练方法**：理解负采样、层次softmax等优化技术
4. **掌握词向量在VLA中的应用**：理解词向量在VLA模型中的具体应用

---


## 1. 词向量的基本原理

### 1.1 什么是词向量

**直观理解**：想象词向量就像给每个词语制作一张"身份证"，这张身份证用数字编码了这个词语的含义。

### 1.2 One-hot编码 vs 词向量

**One-hot编码**：将词语编码为稀疏向量，只有一个元素为1，其他为0。

**词向量**：将词语编码为密集向量，所有元素都是实数，能够捕获词语的语义信息。

**区别示例**：
- One-hot编码："猫" = [0, 0, 1, 0, 0, ...]（只有第3个元素为1）
- 词向量："猫" = [0.2, -0.1, 0.5, 0.3, ...]（所有元素都是实数）

### 1.3 为什么需要词向量

1. **语义捕获**：能够捕获词语的语义信息，语义相似的词语在向量空间中距离较近
2. **计算效率**：将离散的词语转换为连续的数值向量，便于神经网络处理
3. **维度降低**：词向量的维度通常远小于词汇表大小，降低计算复杂度
4. **预训练方法**：可以使用预训练的词向量，提高特征提取质量

### 1.4 词向量的数学表示详解

#### 1.4.1 从基础数学开始

**步骤1：理解词汇表**

词汇表$V$是所有可能词语的集合，大小为$|V|$。例如，如果词汇表包含10000个词语，则$|V| = 10000$。

**步骤2：理解词向量矩阵**

词向量矩阵$W \in \mathbb{R}^{|V| \times d}$是一个$|V| \times d$的矩阵，其中：
- $|V|$是词汇表大小
- $d$是向量维度（通常为100-300）

**步骤3：理解词向量的提取**

对于词语$w_i$（索引为$i$），其词向量为：

$$\mathbf{v}_i = W[i, :]$$

其中$W[i, :]$表示矩阵$W$的第$i$行。

#### 1.4.2 具体计算示例

**示例：提取词向量**

假设：
- 词汇表大小：$|V| = 10000$
- 向量维度：$d = 100$
- 词语"猫"的索引：$i = 3$

**步骤1：获取词向量矩阵**

$$W = \begin{bmatrix}
w_{0,0} & w_{0,1} & \cdots & w_{0,99} \\
w_{1,0} & w_{1,1} & \cdots & w_{1,99} \\
w_{2,0} & w_{2,1} & \cdots & w_{2,99} \\
w_{3,0} & w_{3,1} & \cdots & w_{3,99} \\
\vdots & \vdots & \ddots & \vdots \\
w_{9999,0} & w_{9999,1} & \cdots & w_{9999,99}
\end{bmatrix}$$

**步骤2：提取"猫"的词向量**

$$\mathbf{v}_{猫} = W[3, :] = [w_{3,0}, w_{3,1}, \ldots, w_{3,99}]$$

例如：$\mathbf{v}_{猫} = [0.2, -0.1, 0.5, 0.3, \ldots]$

### 1.5 词向量的可视化

下面我们通过代码可视化词向量的概念：


In [ ]:
# ============================================
# 导入必要的库
# ============================================
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-darkgrid')
print("环境准备完成！")


In [ ]:
# ============================================
# 词向量可视化（示例：10个词语，100维向量）
# ============================================
np.random.seed(42)

# 模拟词向量（10个词语，100维）
vocab_size = 10
embed_dim = 100
words = ['猫', '狗', '鸟', '鱼', '汽车', '飞机', '火车', '船', '桌子', '椅子']

# 生成模拟词向量（语义相似的词语向量相近）
word_vectors = np.random.randn(vocab_size, embed_dim)

# 使语义相似的词语向量相近
# 动物类：猫、狗、鸟、鱼
animal_base = np.random.randn(embed_dim)
word_vectors[0:4] = animal_base + np.random.randn(4, embed_dim) * 0.3

# 交通工具类：汽车、飞机、火车、船
vehicle_base = np.random.randn(embed_dim)
word_vectors[4:8] = vehicle_base + np.random.randn(4, embed_dim) * 0.3

# 家具类：桌子、椅子
furniture_base = np.random.randn(embed_dim)
word_vectors[8:10] = furniture_base + np.random.randn(2, embed_dim) * 0.3

# 使用PCA降维到2D
pca = PCA(n_components=2)
word_vectors_2d = pca.fit_transform(word_vectors)

# 可视化
fig, ax = plt.subplots(figsize=(12, 10))

# 定义颜色（动物=红色，交通工具=蓝色，家具=绿色）
colors = ['red'] * 4 + ['blue'] * 4 + ['green'] * 2

# 绘制词向量
for i, word in enumerate(words):
    ax.scatter(word_vectors_2d[i, 0], word_vectors_2d[i, 1], 
              s=300, c=colors[i], edgecolors='black', linewidths=2, alpha=0.7)
    ax.text(word_vectors_2d[i, 0], word_vectors_2d[i, 1], word, 
           ha='center', va='center', fontsize=12, fontweight='bold')

ax.set_title('词向量可视化（PCA降维到2D）\n语义相似的词语在向量空间中距离较近', 
            fontsize=14, fontweight='bold')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.grid(True, alpha=0.3)

# 添加图例
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', edgecolor='black', label='动物类'),
    Patch(facecolor='blue', edgecolor='black', label='交通工具类'),
    Patch(facecolor='green', edgecolor='black', label='家具类')
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

print("=" * 60)
print("词向量可视化说明：")
print("=" * 60)
print("1. 红色点：动物类（猫、狗、鸟、鱼），在向量空间中聚集在一起")
print("2. 蓝色点：交通工具类（汽车、飞机、火车、船），在向量空间中聚集在一起")
print("3. 绿色点：家具类（桌子、椅子），在向量空间中聚集在一起")
print("4. 语义相似的词语在向量空间中距离较近，这证明了词向量能够捕获语义信息")
print("=" * 60)


## 2. Word2Vec详解

### 2.1 Skip-gram详解

#### 2.1.1 什么是Skip-gram

**直观理解**：想象Skip-gram就像一位"猜谜游戏"的玩家，他看到一个词语（中心词语），然后猜测这个词语周围可能出现的其他词语（上下文词语）。

#### 2.1.2 为什么需要Skip-gram

1. **语义捕获**：通过预测上下文词语，能够捕获词语的语义信息
2. **罕见词语处理**：适合处理罕见词语，因为每个中心词语都有多个上下文词语
3. **训练效率**：训练过程相对简单，计算效率较高

#### 2.1.3 Skip-gram的数学推导详解

#### 2.1.3.1 从基础数学开始

**步骤1：理解条件概率**

条件概率$P(A|B)$表示在事件$B$发生的条件下，事件$A$发生的概率。

**步骤2：理解Skip-gram的目标**

Skip-gram的目标是最大化上下文词语的条件概率：

$$\max \prod_{t=1}^{T} \prod_{-c \leq j \leq c, j \neq 0} P(w_{t+j} | w_t)$$

其中：
- $w_t$是中心词语
- $w_{t+j}$是上下文词语
- $c$是上下文窗口大小（通常为2-5）
- $T$是文本长度

**步骤3：理解softmax归一化**

使用softmax将词向量映射到概率分布：

$$P(w_{t+j} | w_t) = \frac{\exp(\mathbf{v}_{w_{t+j}}^T \mathbf{v}_{w_t})}{\sum_{k=1}^{|V|} \exp(\mathbf{v}_{w_k}^T \mathbf{v}_{w_t})}$$

其中：
- $\mathbf{v}_{w_t}$是中心词语的词向量
- $\mathbf{v}_{w_{t+j}}$是上下文词语的词向量
- $|V|$是词汇表大小

**为什么使用点积？**

点积$\mathbf{v}_{w_{t+j}}^T \mathbf{v}_{w_t}$衡量两个词向量的相似度：
- 如果两个词向量相似（点积大），则概率高
- 如果两个词向量不相似（点积小），则概率低

#### 2.1.3.2 Skip-gram的具体计算示例

**示例：计算Skip-gram的概率**

假设我们有一个句子："猫 是 宠物"，窗口大小$c=1$。

**训练样本**：
- 中心词语："是"（索引$i=5$）
- 上下文词语："猫"（索引$j=3$）、"宠物"（索引$k=10$）

**步骤1：获取词向量**

假设词向量矩阵$W$已经训练好：
- $\mathbf{v}_{是} = W[5, :] = [0.1, 0.2, -0.1, \ldots]$
- $\mathbf{v}_{猫} = W[3, :] = [0.2, -0.1, 0.5, \ldots]$
- $\mathbf{v}_{宠物} = W[10, :] = [0.3, 0.1, 0.4, \ldots]$

**步骤2：计算点积**

对于中心词语"是"和上下文词语"猫"：

$$\mathbf{v}_{猫}^T \mathbf{v}_{是} = 0.1 \times 0.2 + 0.2 \times (-0.1) + (-0.1) \times 0.5 + \ldots = 2.5$$

**步骤3：计算softmax概率**

假设词汇表大小$|V| = 10000$，我们需要计算所有词语的点积，然后使用softmax归一化：

$$P(猫 | 是) = \frac{\exp(2.5)}{\sum_{k=1}^{10000} \exp(\mathbf{v}_{w_k}^T \mathbf{v}_{是})} \approx 0.08$$

**步骤4：理解概率的含义**

概率值越大，说明"猫"在"是"的上下文中出现的可能性越大。

#### 2.1.3.3 Skip-gram的损失函数

Skip-gram的损失函数是负对数似然：

$$\mathcal{L} = -\sum_{t=1}^{T} \sum_{-c \leq j \leq c, j \neq 0} \log P(w_{t+j} | w_t)$$

**损失函数的含义**：
- 损失值越小，说明模型预测越准确
- 通过最小化损失函数，模型学习到更好的词向量

### 2.2 CBOW详解

#### 2.2.1 什么是CBOW

**直观理解**：想象CBOW就像一位"填空游戏"的玩家，他看到上下文词语，然后猜测中间缺失的词语（中心词语）。

#### 2.2.2 CBOW的数学推导详解

CBOW的目标是最大化中心词语的条件概率：

$$\max \prod_{t=1}^{T} P(w_t | w_{t-c}, \ldots, w_{t-1}, w_{t+1}, \ldots, w_{t+c})$$

其中：
- $w_t$是中心词语
- $w_{t-c}, \ldots, w_{t-1}, w_{t+1}, \ldots, w_{t+c}$是上下文词语（排除$w_t$）

**上下文向量的计算**：

$$\mathbf{v}_{context} = \frac{1}{2c} \sum_{-c \leq j \leq c, j \neq 0} \mathbf{v}_{w_{t+j}}$$

**中心词语的概率**：

$$P(w_t | context) = \frac{\exp(\mathbf{v}_{w_t}^T \mathbf{v}_{context})}{\sum_{k=1}^{|V|} \exp(\mathbf{v}_{w_k}^T \mathbf{v}_{context})}$$

#### 2.2.3 CBOW vs Skip-gram

**CBOW的优势**：
- 训练速度更快（因为上下文词语的平均减少了计算量）
- 适合处理常见词语

**Skip-gram的优势**：
- 适合处理罕见词语（因为每个中心词语都有多个上下文词语）
- 通常能够学习到更好的词向量

### 2.3 负采样详解

#### 2.3.1 为什么需要负采样

计算所有词语的softmax计算复杂度为$O(|V|)$，当词汇表很大时（如$|V| = 10000$），计算成本很高。负采样通过只计算正样本和少量负样本，将计算复杂度降低到$O(k+1)$，其中$k$是负样本数量（通常为5-20）。

#### 2.3.2 负采样的数学表示

负采样的目标函数：

$$\max \log \sigma(\mathbf{v}_{w_O}^T \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} [\log \sigma(-\mathbf{v}_{w_i}^T \mathbf{v}_{w_I})]$$

其中：
- $\sigma(x) = \frac{1}{1 + \exp(-x)}$是sigmoid函数
- $w_O$是正样本（上下文词语）
- $w_I$是中心词语
- $w_i$是负样本（随机采样的词语）
- $P_n(w)$是负采样分布（通常为$P_n(w) \propto U(w)^{3/4}$，其中$U(w)$是词语频率）

**负采样分布的含义**：
- 频率高的词语更容易被选为负样本
- 使用$U(w)^{3/4}$而不是$U(w)$，可以减少高频词语的权重

#### 2.3.3 负采样的具体计算示例

**示例：计算负采样的损失**

假设：
- 中心词语："是"（$\mathbf{v}_{是}$）
- 正样本："猫"（$\mathbf{v}_{猫}$）
- 负样本数量：$k = 5$
- 负样本："汽车"、"飞机"、"火车"、"船"、"桌子"

**步骤1：计算正样本的sigmoid**

$$\sigma(\mathbf{v}_{猫}^T \mathbf{v}_{是}) = \sigma(2.5) = \frac{1}{1 + \exp(-2.5)} \approx 0.924$$

**步骤2：计算负样本的sigmoid**

对于每个负样本$w_i$：

$$\sigma(-\mathbf{v}_{w_i}^T \mathbf{v}_{是}) = \frac{1}{1 + \exp(\mathbf{v}_{w_i}^T \mathbf{v}_{是})}$$

例如，对于"汽车"：
- $\mathbf{v}_{汽车}^T \mathbf{v}_{是} = -1.2$
- $\sigma(-\mathbf{v}_{汽车}^T \mathbf{v}_{是}) = \sigma(1.2) = \frac{1}{1 + \exp(-1.2)} \approx 0.769$

**步骤3：计算总损失**

$$\mathcal{L} = -\log(0.924) - \sum_{i=1}^{5} \log(\sigma(-\mathbf{v}_{w_i}^T \mathbf{v}_{是}))$$

**步骤4：理解损失的含义**

损失值越小，说明：
- 正样本（"猫"）与中心词语（"是"）的相似度越高
- 负样本与中心词语的相似度越低

### 2.4 Word2Vec的可视化

下面我们通过代码可视化Word2Vec的训练过程：


In [ ]:
# ============================================
# Skip-gram训练过程可视化（示例：简单句子）
# ============================================
np.random.seed(42)

# 模拟一个简单句子："猫 是 宠物"
sentence = ['猫', '是', '宠物']
vocab = {'猫': 0, '是': 1, '宠物': 2, '汽车': 3, '飞机': 4}
vocab_size = len(vocab)
embed_dim = 10
window_size = 1

# 初始化词向量矩阵（随机初始化）
W = np.random.randn(vocab_size, embed_dim) * 0.1

# 模拟Skip-gram训练（简化版，只训练一个epoch）
def skipgram_step(center_word_idx, context_word_idx, W, learning_rate=0.01):
    """Skip-gram的一个训练步骤（简化版，不使用负采样）"""
    # 获取词向量
    v_center = W[center_word_idx]
    v_context = W[context_word_idx]
    
    # 计算点积
    dot_product = np.dot(v_context, v_center)
    
    # 计算softmax概率（简化版，只考虑当前上下文词语）
    exp_score = np.exp(dot_product)
    # 这里简化处理，实际应该计算所有词语的softmax
    
    # 计算梯度（简化版）
    # 实际梯度计算需要考虑所有词语的softmax
    grad_center = v_context * (1 - exp_score / (exp_score + 1))
    grad_context = v_center * (1 - exp_score / (exp_score + 1))
    
    # 更新词向量
    W[center_word_idx] -= learning_rate * grad_center
    W[context_word_idx] -= learning_rate * grad_context
    
    return W, dot_product

# 生成训练样本
training_samples = []
for i in range(1, len(sentence) - 1):  # 跳过首尾词语
    center_word = sentence[i]
    context_words = [sentence[i-1], sentence[i+1]]
    for context_word in context_words:
        training_samples.append((vocab[center_word], vocab[context_word]))

# 可视化训练过程
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左图：训练前的词向量（PCA降维到2D）
pca = PCA(n_components=2)
W_2d_before = pca.fit_transform(W)

ax = axes[0]
for word, idx in vocab.items():
    if word in sentence:
        ax.scatter(W_2d_before[idx, 0], W_2d_before[idx, 1], 
                  s=300, c='red', edgecolors='black', linewidths=2, alpha=0.7)
        ax.text(W_2d_before[idx, 0], W_2d_before[idx, 1], word, 
               ha='center', va='center', fontsize=12, fontweight='bold')
    else:
        ax.scatter(W_2d_before[idx, 0], W_2d_before[idx, 1], 
                  s=200, c='gray', edgecolors='black', linewidths=1, alpha=0.5)
        ax.text(W_2d_before[idx, 0], W_2d_before[idx, 1], word, 
               ha='center', va='center', fontsize=10)

ax.set_title('训练前的词向量（随机初始化）', fontsize=12, fontweight='bold')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.grid(True, alpha=0.3)

# 执行几个训练步骤
for _ in range(10):
    for center_idx, context_idx in training_samples:
        W, _ = skipgram_step(center_idx, context_idx, W, learning_rate=0.1)

# 右图：训练后的词向量（PCA降维到2D）
W_2d_after = pca.fit_transform(W)

ax = axes[1]
for word, idx in vocab.items():
    if word in sentence:
        ax.scatter(W_2d_after[idx, 0], W_2d_after[idx, 1], 
                  s=300, c='green', edgecolors='black', linewidths=2, alpha=0.7)
        ax.text(W_2d_after[idx, 0], W_2d_after[idx, 1], word, 
               ha='center', va='center', fontsize=12, fontweight='bold')
    else:
        ax.scatter(W_2d_after[idx, 0], W_2d_after[idx, 1], 
                  s=200, c='gray', edgecolors='black', linewidths=1, alpha=0.5)
        ax.text(W_2d_after[idx, 0], W_2d_after[idx, 1], word, 
               ha='center', va='center', fontsize=10)

ax.set_title('训练后的词向量（经过Skip-gram训练）', fontsize=12, fontweight='bold')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("=" * 60)
print("Skip-gram训练过程可视化说明：")
print("=" * 60)
print("1. 左图：训练前的词向量（随机初始化），词语在向量空间中随机分布")
print("2. 右图：训练后的词向量（经过Skip-gram训练），句子中的词语（红色/绿色）在向量空间中更接近")
print("3. Skip-gram通过预测上下文词语，使语义相关的词语在向量空间中距离更近")
print("=" * 60)


## 3. 词向量在VLA中的应用

### 3.1 语言指令编码

在VLA中，词向量用于将语言指令中的词语转换为向量表示，例如：
- 将"拿起"转换为向量$\mathbf{v}_{拿起}$
- 将"杯子"转换为向量$\mathbf{v}_{杯子}$
- 将"桌子上"转换为向量$\mathbf{v}_{桌子上}$

### 3.2 语义理解

词向量的语义信息能够帮助VLA模型理解语言指令中词语的含义，例如：
- "拿起"和"抓取"语义相似，它们的词向量在向量空间中距离较近
- "杯子"和"水杯"语义相似，它们的词向量在向量空间中距离较近

### 3.3 预训练方法

词向量可以用于VLA的预训练，通过大规模文本数据预训练词向量，然后在VLA任务上进行微调。

### 3.4 在VLA中的具体应用示例

**示例：理解"拿起桌子上的杯子"**

如果语言指令是"拿起桌子上的杯子"，VLA模型需要：
1. 通过词向量将每个词语转换为向量表示
2. 使用语言编码器（如BERT）融合词语向量，得到句子向量
3. 理解指令的意图（"拿起"）、对象（"杯子"）、位置（"桌子上"）
4. 结合视觉理解结果生成相应的动作序列

---

## 4. 总结

### 4.1 词向量的核心思想

1. **分布式表示**：将词语编码为密集向量，捕获词语的语义信息
2. **语义相似性**：语义相似的词语在向量空间中距离较近
3. **训练方法**：使用Word2Vec等方法训练词向量
4. **预训练模型**：可以使用预训练的词向量，提高特征提取质量

### 4.2 词向量的优势

1. **语义捕获**：能够捕获词语的语义信息
2. **计算效率**：将离散的词语转换为连续的数值向量，便于神经网络处理
3. **维度降低**：词向量的维度通常远小于词汇表大小
4. **预训练方法**：可以使用预训练的词向量，提高特征提取质量

### 4.3 在VLA中的意义

词向量是VLA语言理解的基础，它提供了词语的数值表示，使得计算机能够处理和理解自然语言。词向量的语义信息能够帮助VLA模型理解语言指令中词语的含义，从而更好地理解指令的意图，生成更准确的动作序列。

---

**文档完成时间**：2025-01-27  
**文档版本**：v1.0  
**维护者**：AI助手

**相关文档**：
- 父目录：文本特征提取详解（见../文本特征提取详解.ipynb）
的01-